In [1]:
%%javascript
IPython.notebook.kernel.execute('nb_name = "' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [2]:
%load_ext autoreload
%autoreload
from IPython.display import clear_output

import os
import pdb
import _pickle as cPickle
import time
import subprocess
import glob

import random
import numpy as np
import pandas as pd
import tensorflow as tf

from collections import defaultdict, Counter
from ncrp import Topic, Doc, init, sample, get_perplexity, get_topic_specialization, get_hierarchical_affinities, get_freq_tokens_ncrp, get_docs, get_sum_cnt_words
from configure import get_config

# load config & data 

In [3]:
config = get_config(nb_name)
np.random.seed(config.seed)
random.seed(config.seed)

In [4]:
instances_train, instances_dev, instances_test, word_to_idx, idx_to_word, bow_idxs = cPickle.load(open(config.path_data,'rb'))

In [5]:
config.n_doc = len(instances_train)
config.n_vocab = len(bow_idxs)
config.n_doc, config.n_vocab

(9006, 1995)

# run

## initialize log

In [ ]:
checkpoint = []
ppl_min = np.inf
epoch = 0

cmd_rm = 'rm -r %s' % config.dir_model
res = subprocess.call(cmd_rm.split())
cmd_mk = 'mkdir %s' % config.dir_model
res = subprocess.call(cmd_mk.split())

log_df = pd.DataFrame(columns=pd.MultiIndex.from_tuples(
                    list(zip(*[['','','','TRAIN:','VALID:','TEST:','SPEC:', '', '', 'HIER:', ''],
                            ['Time','Ep','Ct','PPL','PPL', 'PPL','1', '2', '3', 'CHILD', 'OTHER']]))))

def update_checkpoint(config, checkpoint, epoch):
    checkpoint.append(config.path_model + '-%i' % epoch)
    if len(checkpoint) > config.max_to_keep:
        path_model = checkpoint.pop(0)
        for p in glob.glob(path_model):
            os.remove(p)
    cPickle.dump(checkpoint, open(config.path_checkpoint, 'wb'))

## initialize data

In [ ]:
topic_root = Topic(idx='0', sibling_idx=0, parent=None, depth=0, config=config)
train_docs = get_docs(instances_train, config)
dev_docs = get_docs(instances_dev, config)
test_docs = get_docs(instances_test, config)
init(train_docs, dev_docs, test_docs, topic_root)

0 0 0 

## run

In [ ]:
time_start = time.time()
while epoch < config.n_epochs:
    sample(train_docs, dev_docs, test_docs, topic_root)
    ppl_train = get_perplexity(train_docs, topic_root)
    ppl_dev = get_perplexity(dev_docs, topic_root)
    
    if ppl_dev < ppl_min:
        ppl_min = ppl_dev
        ppl_test = get_perplexity(test_docs, topic_root)
        cPickle.dump([test_docs, topic_root], open(config.path_model + '-%i'%epoch, 'wb'))
        update_checkpoint(config, checkpoint, epoch)
        
    depth_spec = get_topic_specialization(test_docs, topic_root)
    hierarchical_affinities = get_hierarchical_affinities(topic_root)
    
    clear_output()
    time_log = int(time.time() - time_start)
    time_start = time.time()
    log_series = pd.Series([time_log, epoch, 0, \
            '%.0f'%ppl_train, '%.0f'%ppl_dev, '%.0f'%ppl_test, \
            '%.2f'%depth_spec[1], '%.2f'%depth_spec[2], '%.2f'%depth_spec[3], \
            '%.2f'%hierarchical_affinities[0], '%.2f'%hierarchical_affinities[1]],
            index=log_df.columns)
    log_df.loc[epoch] = log_series    
    display(log_df)
    get_freq_tokens_ncrp(topic_root, idx_to_word, bow_idxs)
    
    cPickle.dump(log_df, open(config.path_log, 'wb'))
    epoch += 1

TRAIN: VALID: TEST: SPEC:             HIER:      
  Time Ep Ct    PPL    PPL   PPL     1     2     3 CHILD OTHER
0  360  0  0   1038   1243  1076  0.03  0.30  0.45  0.92  0.92
1  646  1  0   1026   1216  1059  0.03  0.38  0.45  0.96  0.96
2  699  2  0   1017   1191  1046  0.03  0.40  0.45  0.97  0.97
3  679  3  0   1009   1171  1036  0.03  0.42  0.45  0.97  0.97
4  657  4  0   1004   1155  1029  0.03  0.43  0.45  0.97  0.97
5  700  5  0    999   1143  1023  0.03  0.43  0.45  0.97  0.97
6  709  6  0    993   1128  1016  0.03  0.45  0.45  0.97  0.97
7  718  7  0    985   1117  1007  0.03  0.45  0.45  0.97  0.97
8  741  8  0    977   1105   999  0.03  0.45  0.45  0.96  0.96

 0 9006 604786.0 use write one get article know like make go think
   0-1 5754 65301.0 god space jesus christian launch believe truth faith people satellite
     0-1-1 5754 3082.0 imagine write defense lab steve fall involve small distance mass
   0-2 707 13670.0 pt vs la period play gm win van pp team
     0-2-1 707 506.0 hr gm sin os east los cup total apr leafs
   0-3 2543 7141.0 rg oo ripem eus pm sequence unit echo mr ground
     0-3-1 2543 999.0 hus adapter helmet star thread piece box like advance reader
   0-4 2 24.0 tire cold longer therefore tend exist specify oil manual manufacturer
     0-4-1 2 237.0 bh db mov cs byte pop al east west ax
0 0 